In [69]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [70]:
data2014 = pd.read_csv("finalized/2014-scores.csv")
data2016 = pd.read_csv("finalized/2016-scores.csv")
data2018 = pd.read_csv("finalized/2018-scores.csv")

df2018 = data2018[data2018.columns.difference(['census_tract'])]
df2016 = data2016[data2016.columns.difference(['census_tract'])]
df2014 = data2014[data2014.columns.difference(['census_tract'])]

In [71]:
df2018['year'] = 2018
df2016['year'] = 2016
df2014['year'] = 2014

In [72]:
df2018.head()

,associates,bachelors,car,diff_countyHouse,diff_countyRent,diff_stateHouse,diff_stateRent,full_name,health_ins,house_val,museums,pharmacies,population,poverty,score,self_income,year
0,-1.211784,-1.253158,-1.226062,-0.637187,-0.476067,-0.591463,0.118601,"Census Tract 1, Atlantic County, New Jersey",-1.198725,-0.945236,0.312933,-0.308789,-1.226062,0.901790,13.435532,-1.038599,2018
1,0.709789,-0.520335,1.311203,1.009952,0.500331,-0.684851,-0.515725,"Census Tract 1, Essex County, New Jersey",1.340113,-0.540658,-0.125130,1.764868,1.311203,1.532821,41.527496,-0.371905,2018
2,0.017033,0.875323,0.992570,-0.008897,0.048565,-0.684851,-0.515725,"Census Tract 1, Hudson County, New Jersey",1.021282,0.028898,-0.709213,0.579921,0.992570,0.155624,37.072678,-0.421658,2018
3,-0.733453,-1.115239,-0.615080,-0.722091,-0.476067,-0.684851,-0.083844,"Census Tract 1, Mercer County, New Jersey",-0.587365,-1.241689,1.335079,-0.308789,-0.615080,1.083350,19.201547,-1.366971,2018
4,2.466419,0.780633,1.719419,-0.382475,3.633549,-0.428033,-0.029859,"Census Tract 1, Middlesex County, New Jersey",1.748581,-0.072119,-0.417171,0.283684,1.719419,0.075914,48.512330,1.130645,2018


In [73]:
le = preprocessing.LabelEncoder()
df2018['full_name'] = le.fit_transform(df2018['full_name'])
df2016['full_name'] = le.fit_transform(df2016['full_name'])
df2014['full_name'] = le.fit_transform(df2014['full_name'])


In [74]:
df2018 = df2018[['full_name', 'year', 'associates', 'bachelors', "car", "diff_countyHouse", "diff_countyRent", "diff_stateHouse", "diff_stateRent", "health_ins", "house_val", "museums", "pharmacies", "population", "poverty", "self_income", "score"]]
df2016 = df2016[['full_name', 'year', 'associates', 'bachelors', "car", "diff_countyHouse", "diff_countyRent", "diff_stateHouse", "diff_stateRent", "health_ins", "house_val", "museums", "pharmacies", "population", "poverty", "self_income", "score"]]
df2014 = df2014[['full_name', 'year', 'associates', 'bachelors', "car", "diff_countyHouse", "diff_countyRent", "diff_stateHouse", "diff_stateRent", "health_ins", "house_val", "museums", "pharmacies", "population", "poverty", "self_income", "score"]]

df2014 = df2014.drop([2006, 2007])

In [76]:
print(df2016.columns)
print(df2014.shape)

Index(['full_name', 'year', 'associates', 'bachelors', 'car',
       'diff_countyHouse', 'diff_countyRent', 'diff_stateHouse',
       'diff_stateRent', 'health_ins', 'house_val', 'museums', 'pharmacies',
       'population', 'poverty', 'self_income', 'score'],
      dtype='object')
(2006, 17)


In [78]:
np.array([df2018.to_numpy(), df2016.to_numpy(), df2014.to_numpy()])

array([[[ 0.00000000e+00,  2.01800000e+03, -1.21178404e+00, ...,
          9.01790290e-01, -1.03859919e+00,  1.34355320e+01],
        [ 1.00000000e+00,  2.01800000e+03,  7.09788532e-01, ...,
          1.53282142e+00, -3.71904781e-01,  4.15274962e+01],
        [ 2.00000000e+00,  2.01800000e+03,  1.70327540e-02, ...,
          1.55623660e-01, -4.21658095e-01,  3.70726775e+01],
        ...,
        [ 2.00300000e+03,  2.01800000e+03, -1.01385382e+00, ...,
          1.83467132e-02, -1.07840184e+00,  1.42794998e+01],
        [ 2.00400000e+03,  2.01800000e+03, -1.64887995e+00, ...,
         -9.97945522e-01, -1.49632968e+00,  7.48487142e-01],
        [ 2.00500000e+03,  2.01800000e+03, -1.64887995e+00, ...,
         -9.97945522e-01, -1.49632968e+00,  0.00000000e+00]],

       [[ 0.00000000e+00,  2.01600000e+03, -1.12444844e+00, ...,
          8.44989594e-01, -9.41715506e-01,  1.50445063e+01],
        [ 1.00000000e+00,  2.01600000e+03,  7.03955591e-02, ...,
          1.80408349e+00, -4.86863928e